In [1]:
%pylab inline
import pandas as pd

from dataManager import DataManager
from model import ModelManager as manager

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as mse

import plotly.graph_objects as go

Populating the interactive namespace from numpy and matplotlib


In [52]:
data = DataManager().sales_ref_month_sin_ventas_mayores()
data['DATE'] = pd.to_datetime(data['ANIO'].astype(str) + '/' + data['MES'].astype(str))
data.CANTIDAD = data.CANTIDAD.fillna(0)
data = data.groupby(['REF','DATE']).sum()

In [55]:
data

ANIO  MES        PRECIO  SUBTOTAL  DESCUENTO(%)  \
REF           DATE                                                          
A01040:00005: 2019-02-01  2019    2  1.452102e+05   2032943      0.650000   
BR0002:00193: 2020-10-01  6060   30  9.783005e+05   1291109      1.616042   
              2020-11-01  4040   22  6.505155e+05   1629498      0.822970   
              2020-12-01  6060   36  9.725640e+05    972564      1.177835   
              2021-01-01  2021    1  3.255930e+05    651186      0.417836   
...                        ...  ...           ...       ...           ...   
XZ0993:00100: 2020-07-01  2020    7  1.640030e+05    164003      0.000000   
              2020-10-01  2020   10  5.466770e+05    546677      0.700000   
              2021-02-01  4042    4  1.093355e+06   1093355      1.400001   
              2021-03-01  2021    3  5.466780e+05    546678      0.600000   
              2021-04-01  6063   12  1.733602e+06   1733602      1.607510   

                           TOTAL  CANTIDAD   ALTO     AREA  PUESTOS  \
REF           DATE                                                    
A01040:00005: 2019-02-01  711530        14   77.0   1880.0      1.0   
BR0002:00193: 2020-10-01  595295         4  540.0   7749.0      0.0   
              2020-11-01  939498         5  360.0   5166.0      0.0   
              2020-12-01  590724         3  540.0   7749.0      0.0   
              2021-01-01  379026         2  180.0   2583.0      0.0   
...                          ...       ...    ...      ...      ...   
XZ0993:00100: 2020-07-01  164003         1   44.0   5000.0      0.0   
              2020-10-01  164003         1   44.0   5000.0      0.0   
              2021-02-01  328006         2   88.0  10000.0      0.0   
              2021-03-01  218671         1   44.0   5000.0      0.0   
              2021-04-01  808612         3  132.0  15000.0      0.0   

                          Factor covid  
REF           DATE                      
A01040:00005: 2019-02-01             0  
BR0002:00193: 2020-10-01             3  
              2020-11-01             2  
              2020-12-01             6  
              2021-01-01             2  
...                                ...  
XZ0993:00100: 2020-07-01             1  
              2020-10-01             1  
              2021-02-01             2  
              2021-03-01             1  
              2021-04-01             3  

[9787 rows x 11 columns]

In [51]:
df = sales_prod.pivot_table(index='REF',columns=['DATE','TIENDA'],values='CANTIDAD',aggfunc='sum').reset_index()
df = df.fillna(0)
df[['REF','2019-03']]

DATE              REF        2019-03                                         \
TIENDA                ADMINISTRACION PAGINA WEB FIOTTI PUNTO DE VENTA AV 68   
0       A01040:00005:            0.0               0.0                 24.0   
1       BR0002:00193:            0.0               0.0                  0.0   
2       BR0003:00194:            0.0               0.0                  0.0   
3       BR0004:00193:            0.0               0.0                  0.0   
4       BR0005:00193:            0.0               0.0                  0.0   
...               ...            ...               ...                  ...   
1248    XZ0986:00005:            0.0               0.0                  0.0   
1249    XZ0987:00024:            0.0               1.0                  1.0   
1250    XZ0988:00005:            0.0               1.0                  5.0   
1251    XZ0989:00024:            1.0               0.0                  0.0   
1252    XZ0993:00100:            0.0               0.0                  0.0   

DATE                                                   \
TIENDA PUNTO DE VENTA BUCARAMANGA PUNTO DE VENTA CALI   
0                             0.0                 0.0   
1                             0.0                 0.0   
2                             0.0                 0.0   
3                             0.0                 0.0   
4                             0.0                 0.0   
...                           ...                 ...   
1248                          0.0                 0.0   
1249                          0.0                 1.0   
1250                          1.0                 0.0   
1251                          0.0                 0.0   
1252                          0.0                 0.0   

DATE                                                         \
TIENDA PUNTO DE VENTA CALLE 134 PUNTO DE VENTA CENTRO MAYOR   
0                           0.0                         0.0   
1                           0.0                         0.0   
2                           0.0                         0.0   
3                           0.0                         0.0   
4                           0.0                         0.0   
...                         ...                         ...   
1248                        1.0                         0.0   
1249                        0.0                         0.0   
1250                        0.0                         0.0   
1251                        0.0                         0.0   
1252                        0.0                         0.0   

DATE                                                         \
TIENDA PUNTO DE VENTA LAURELES PUNTO DE VENTA PLAZA CENTRAL   
0                          0.0                          0.0   
1                          0.0                          0.0   
2                          0.0                          0.0   
3                          0.0                          0.0   
4                          0.0                          0.0   
...                        ...                          ...   
1248                       0.0                          0.0   
1249                       0.0                          0.0   
1250                       0.0                          0.0   
1251                       0.0                          0.0   
1252                       0.0                          0.0   

DATE                                                         \
TIENDA PUNTO DE VENTA PLAZA IMPERIAL PUNTO DE VENTA POBLADO   
0                                0.0                    0.0   
1                                0.0                    0.0   
2                                0.0                    0.0   
3                                0.0                    0.0   
4                                0.0                    0.0   
...                              ...                    ...   
1248                             0.0                    0.0   
1249                   

In [ ]:
df = data.pivot_table(index='REF',columns=['DATE','TIENDA'],values='CANTIDAD',aggfunc='sum').reset_index()
df = pd.melt(df,id_vars='REF')

df = df.sort_values(['REF','DATE'])
df = df.rename(columns={'value':'CANTIDAD'})
df = df.reset_index(drop=True).fillna(0)
df.CANTIDAD = df.CANTIDAD.astype(int)

df = df.merge(data,on=['REF'],how='left')
#df = df.drop(columns=['CANTIDAD_y','DATE_YM_y']).rename(columns={'CANTIDAD_x':'CANTIDAD','DATE_YM_x':'DATE','TIENDA_x':'TIENDA'})

In [ ]:
num_var=['AREA','ALTO','DESCUENTO(%)','PRECIO','CANTIDAD']
x_num=df[num_var[:-1]].astype('float')

cat_var=['MES','Factor covid','PUESTOS','COLOR_POS','CATEGORIA','SUBCATEGORIA_POS','VIGENCIA','ORIGEN','ESTILO','MATERIAL_POS','ACABADO','TIENDA']
x_cat=df[cat_var].astype('category')
x_cat_dummies=pd.get_dummies(x_cat)

y = df['CANTIDAD']

scaler = MinMaxScaler()
x_num_norm = scaler.fit_transform(x_num)
x = np.append(x_num_norm,x_cat_dummies,axis=1)

#split data till januar 2021
index = df[(df.ANIO==2021)].index[0]

x_train = x[:index-1]
y_train = y[:index-1]
x_test = x[index-1:]
y_test = y[index-1:]

In [ ]:
model = GradientBoostingRegressor(**{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 300})
model.fit(x_train,y_train)

In [ ]:
df['PREDICTED'] = model.predict(np.concatenate([x_train,x_test],axis=0))

In [ ]:
aux = df.query('REF=="A01040:00005:" & TIENDA=="PUNTO DE VENTA AV 68"')
aux[['REF','DATE','CANTIDAD','PREDICTED','TIENDA']]

In [ ]:
fig = go.Figure()
fig.add_scatter(x=aux['DATE'], y=aux['PREDICTED'], mode='lines', name='Valores predichos')
fig.add_scatter(x=aux['DATE'], y=aux['CANTIDAD'], mode='lines', name='Valores reales')